In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

df = pd.read_excel(r"C:\Users\Semanur\Downloads\20240130062437674.XLSX", usecols=["Title", "Claims", "Application Date", "CPC"])

df['Application Date'] = pd.to_datetime(df['Application Date'], errors='coerce')

df['Year'] = df['Application Date'].dt.year
cpc_code = input("Lütfen CPC kodunu girin: ")

filtered_df = df[df['CPC'].str.contains(cpc_code, na=False, case=False)]

filtered_yearly_counts = filtered_df.groupby('Year').size().reset_index(name='Patent Count')
filtered_yearly_counts['Cumulative Patents'] = filtered_yearly_counts['Patent Count'].cumsum()

print(f"CPC kodu '{cpc_code}' için yıl bazında patent sayıları:")
print(filtered_yearly_counts)

years = np.array(filtered_yearly_counts['Year'])
cumulative = np.array(filtered_yearly_counts['Cumulative Patents'])

# S CURVE
def logistic_growth(x, L, k, x0):
    return L / (1 + np.exp(-k * (x - x0)))

popt, pcov = curve_fit(logistic_growth, years, cumulative, bounds=([0, 0, min(years)], [max(cumulative)*1.5, 1, max(years)+50]))

x_fit = np.linspace(min(years), max(years)+50, 1000)
y_fit = logistic_growth(x_fit, *popt)

plt.scatter(years, cumulative, label='Veri')
plt.plot(x_fit, y_fit, label='Lojistik büyüme tahmini', color='red')
plt.xlabel('Yıl')
plt.ylabel('Kümülatif Patent Sayısı')
plt.legend()
plt.show()

print("L =", popt[0])
print("k =", popt[1])
print("x0 =", popt[2])